# Module Installation and Uninstallation

- [Install/Uninstall a module onto the system -- `make modules_install`](#install_uninstall)
- [Load a module into memory, or unload a module from memory -- `insmod` and `rmmod`](#insmod_rmmod)
- [Module dependency and solution -- `modprobe` and `depmod`](dependency_resolution_modprobe.ipynb)

Let's reuse the kernel module generated from project [Export Kernel Symbols](export_symbol/export_kernel_symbols.ipynb):

In [1]:
make | grep '\.ko$'

  LD [M]  /home/clymber/Studio/kerdevel/export_symbol/hello_api.ko
  LD [M]  /home/clymber/Studio/kerdevel/export_symbol/hello_feature.ko


In [2]:
modinfo hello_api.ko
modinfo hello_feature.ko

filename:       /home/clymber/Studio/kerdevel/export_symbol/hello_api.ko
description:    A hello world to export kernel symbols.
author:         Clymber Loong
license:        GPL
srcversion:     A7004390F0CCB7F5838113A
depends:        
retpoline:      Y
name:           hello_api
vermagic:       6.1.29 SMP preempt mod_unload modversions 
filename:       /home/clymber/Studio/kerdevel/export_symbol/hello_feature.ko
description:    A demo to invoke symbols that other modules exported.
author:         Clymber Loong
license:        GPL
srcversion:     340E072184795092C2B1DC1
depends:        hello_api
retpoline:      Y
name:           hello_feature
vermagic:       6.1.29 SMP preempt mod_unload modversions 


From the outputs of commands above, we can see that now we have two kernel modules: *hello_features.ko* and *hello_api.ko*. And according to the output "depends:        hello_api", we know the the former one depends on the latter.

## <a id="install_uninstall"> 1. Module Installation and Uninstallation.</a>

Now we are not talking about how to load a kernel module from an external filesystem(such as a hard drive) into the kernel runtime. We are exploring how to inatall a newly built module, like *hello_api.ko*, into Linux standard filesystem hierarchy. According [Filesystem Hierarchy Standard](https://refspecs.linuxfoundation.org/FHS_3.0/fhs/index.html), and a little exploration on the distro I currently use - Ubuntu 22.04, the newly compiled modules should be install into */lib/modules/&lt;kernel release&gt;*:

In [4]:
uname -r # get kernel release
ls -1 /lib/modules

6.1.29
5.19.0-41-generic
6.1.29


### 1.1 Install through the *Makefile*

If we have the module source tree at hand with a propriate *Makefile*, this is an easy and recommendable way to install a module, just employ the command: `make modules_install`. It handles the toiling work: where to install the module,  how to generate the dependency information, and how to sign the signation, etc. Here is my *Makefile*:

In [6]:
cat Makefile

obj-m              = hello_api.o
hello_api-objs     = hello_api_inexit.o

KERN_BUILD := /lib/modules/$(shell uname -r)/build
MODULE_DIR := $(abspath $(dir $(lastword $(MAKEFILE_LIST))))

modules modules_install clean:
	make -C $(KERN_BUILD) M=$(MODULE_DIR) $@

hello_api-objs += hello_api_test.o

obj-m += hello_feature.o


On the output above, "obj-m = hello_api.o" and "obj-m += hello_feature.o" shows that *hello_api.ko* and *hello_feature.ko* will be installed if we perform command `make modules_install`. Let's have a try:

In [19]:
sudo make modules_install

make -C /lib/modules/6.1.29/build M=/home/clymber/Studio/kerdevel/export_symbol modules_install
make[1]: Entering directory '/home/clymber/Studio/linux-6.1.29'
  INSTALL /lib/modules/6.1.29/extra/hello_api.ko
  SIGN    /lib/modules/6.1.29/extra/hello_api.ko
  INSTALL /lib/modules/6.1.29/extra/hello_feature.ko
  SIGN    /lib/modules/6.1.29/extra/hello_feature.ko
  DEPMOD  /lib/modules/6.1.29
make[1]: Leaving directory '/home/clymber/Studio/linux-6.1.29'


Great! Now let's check the result:

In [27]:
# check where were the modules install:
find "/lib/modules/$(uname -r)" -name 'hello_*.ko'

/lib/modules/6.1.29/extra/hello_api.ko
/lib/modules/6.1.29/extra/hello_feature.ko


In [28]:
# check the module dependcy:
grep 'hello_' "/lib/modules/$(uname -r)/modules.dep"

extra/hello_api.ko:
extra/hello_feature.ko: extra/hello_api.ko


In [30]:
# check the signature:
modinfo --field signature hello_api

BD:FC:BC:EE:9C:E4:55:7E:0A:B7:FE:03:91:72:55:6B:76:1B:20:75:
		B7:F4:DA:B9:10:2B:A4:49:C8:50:6C:E3:2A:CA:07:F7:73:02:C6:92:
		A1:54:16:7D:9D:BA:5D:E0:8A:71:C5:65:E8:43:62:B2:27:8B:F8:C1:
		B6:25:11:47:17:7A:3F:27:72:DF:4F:E5:40:11:23:69:68:D9:E8:B8:
		87:66:20:97:5E:E7:76:77:6A:95:5D:40:AF:2B:39:0C:39:9F:B0:62:
		3B:B9:DE:A5:AA:D5:9D:58:F8:53:28:15:F8:96:4C:2E:C7:4A:21:C6:
		2D:F0:3A:B4:71:87:87:BC:4D:7D:AE:06:85:BA:AC:48:84:B0:CA:8B:
		54:0B:68:B2:8C:00:C5:89:C1:E7:00:33:F1:73:52:30:43:54:32:0B:
		44:F3:20:8C:C1:F9:88:F6:FD:CA:DB:D7:9F:28:F8:6D:41:5C:14:D6:
		FB:89:91:EA:BF:00:06:A3:38:96:98:6C:A2:30:1B:CE:08:E3:77:DF:
		32:14:51:7F:D6:EE:FE:6A:54:0D:88:9B:32:9B:4A:2D:87:14:67:10:
		B4:5F:92:31:30:3D:2C:2E:0C:65:F6:E9:E6:A5:27:47:4B:9A:5A:5E:
		D5:04:4C:D6:2F:43:AA:D4:DF:AC:38:5B:D5:6A:79:BC:44:26:4C:47:
		12:3C:9C:03:36:E1:08:90:1D:34:62:82:8A:C5:E3:30:32:0E:C9:D0:
		79:E3:B5:14:63:48:35:C0:81:E5:5D:53:45:F5:92:54:4B:01:AA:EF:
		F9:8C:3E:FF:17:1A:3B:A5:3A:92:BF:C0:9C:4F:9E:1A:5A:7E:7

## 1.2 Uninstall a kernel module

I don't know whether there's a standard way to install a kernel module yet. Let's remove it manually this time.
- step 1: remove the module file from the system.
- step 2: re-generate the dependency information.

In [31]:
find "/lib/modules/$(uname -r)" -name 'hello_*.ko' -exec sudo rm '{}' \;
sudo depmod -a

In [33]:
# check the module dependcy:
grep 'hello_' "/lib/modules/$(uname -r)/modules.dep" || echo 'not found'

not found


## 1.3 Install module manually

If we don't have the module source tree, or if we only want to install `hello_api.ko` but not `hello_feature.ko`, we can install the module manually. The steps are simple:  
- step 1: copy the module the targeted directory, for example, */lib/modules/\$(uname -r)*.   
- step 2 (optionally): sign the signature for the module(I have never tried it yet).  
- step 3: generate dependency information in the file */lib/modules/\$(uname -r)/modules.dep*.

In [35]:
sudo cp "hello_api.ko" "hello_feature.ko" "/lib/modules/$(uname -r)/extra/"
sudo depmod -a

In [36]:
# check the module dependcy:
grep 'hello_' "/lib/modules/$(uname -r)/modules.dep" || echo 'not found'

extra/hello_api.ko:
extra/hello_feature.ko: extra/hello_api.ko


## 2. <a id="insmod_rmmod"> Load/unload a module into/from memory -- `insmod` and `rmmod` </a>

In [37]:
insmod --help

Usage:
	insmod [options] filename [args]
Options:
	-V, --version     show version
	-h, --help        show this help


In [38]:
rmmod --help

Usage:
	rmmod [options] modulename ...
Options:
	-f, --force       forces a module unload and may crash your
	                  machine. This requires Forced Module Removal
	                  option in your kernel. DANGEROUS
	-s, --syslog      print to syslog, not stderr
	-v, --verbose     enables more messages
	-V, --version     show version
	-h, --help        show this help


### 2.1 Load a module with `insmod`

In [47]:
sudo dmesg -C
sudo insmod hello_api.ko; sudo dmesg -c
sudo insmod hello_feature.ko; sudo dmesg -c

[14831.622611] [hello_api] Hello, World!
[14831.688881] [hello_feature] Hello, World!


In [48]:
lsmod | grep 'hello_'

hello_feature          16384  0
hello_api              16384  1 hello_feature


The `lsmod` command shows that two modules are successfully loaded into memory.

### 2.2 Unload a module with `rmmod`

In [49]:
sudo rmmod hello_feature; sudo dmesg -c
sudo rmmod hello_api; sudo dmesg -c

[14831.688883] [hello_api] this is a test message.
[14939.069633] [hello_feature] Bye Bye, World!
[14939.167012] [hello_api] Bye bye, World!


In [50]:
lsmod | grep 'hello_' || echo 'not found'

not found


### 2.3 The dependency issues

`insmod` or `rmmod` will not handle and dependcy issues.

In [45]:
sudo modinfo --field depends hello_feature

hello_api


The output above shows that module *hello_feature* depends on *hello_api*, so if we try to load *hello_feature* before *hello_api*, or if we try to unload *hello_api* while *hello_feature* is still in the memory, we will fail:

In [46]:
sudo insmod hello_feature.ko; sudo dmesg -c

insmod: ERROR: could not insert module hello_feature.ko: Unknown symbol in module
[14797.712189] hello_feature: Unknown symbol api_test (err -2)


In [53]:
# load
sudo insmod hello_api.ko; sudo dmesg -c
sudo insmod hello_feature.ko; sudo dmesg -c

# unload
sudo rmmod hello_api.ko; sudo dmesg -c

[15095.523144] [hello_api] Hello, World!
[15095.588409] [hello_feature] Hello, World!
rmmod: ERROR: Module hello_api is in use by: hello_feature


In [52]:

sudo rmmod hello_api; sudo dmesg -c

[15085.534347] [hello_api] Bye bye, World!


If we don't want to solve the dependency toil, use `modprobe` instead of `insmod/rmmod`: [Module dependency and solution -- `modprobe` and `depmod`](dependency_resolution_modprobe.ipynb)

## Clean up

In [55]:
if lsmod | grep -q 'hello_feature'; then
    sudo rmmod hello_feature
fi

if lsmod | grep -q 'hello_api'; then
    sudo rmmod hello_api
fi

make clean

make -C /lib/modules/6.1.29/build M=/home/clymber/Studio/kerdevel/export_symbol clean
make[1]: Entering directory '/home/clymber/Studio/linux-6.1.29'
make[1]: Leaving directory '/home/clymber/Studio/linux-6.1.29'
